In [1]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from nltk.stem import SnowballStemmer as sno
# setofstopwords=set(stopwords.words('english'))
from sklearn.model_selection import train_test_split

In [22]:
reviews=pd.read_csv('data/amazon_reviews_us_Apparel_v1_00.tsv.gz.tsv',sep='\t')

In [23]:
reviews['label']=(reviews['verified_purchase']=='y').astype(int)

In [24]:
reviews.groupby('verified_purchase').count()

,star_rating,helpful_votes,total_votes,vine,review_headline,review_body,review_date,label
verified_purchase,,,,,,,,
n,593552,593552,593552,593552,593544,593491,593532,593552
y,5312781,5312781,5312781,5312781,5312734,5312053,5312680,5312781


In [31]:
samples=reviews[:5000]

In [32]:
samples.groupby('verified_purchase').count()

,star_rating,helpful_votes,total_votes,vine,review_headline,review_body,review_date,label
verified_purchase,,,,,,,,
n,1754,1754,1754,1754,1754,1754,1754,1754
y,3246,3246,3246,3246,3246,3245,3246,3246


In [58]:
undersample=samples[samples['label']==1].sample(1754)

In [59]:
undersample=pd.concat([undersample,samples[samples['label']==0]])
undersample = undersample.sample(frac=1,replace=False).reset_index(drop=True)

In [60]:
undersample.groupby('verified_purchase').count()

,star_rating,helpful_votes,total_votes,vine,review_headline,review_body,review_date,label
verified_purchase,,,,,,,,
n,1754,1754,1754,1754,1754,1754,1754,1754
y,1754,1754,1754,1754,1754,1754,1754,1754


In [61]:
selected_features=[ 'review_body','label']

In [62]:
# reviews['review_body']=reviews['review_body'].str.lower()

In [63]:
selected_reviews=undersample[selected_features]

In [64]:
selected_reviews['review_body']=selected_reviews['review_body'].str.replace(r'([^a-z\s]+)','')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
selected_reviews['review_body'].head()

0    these were really cute online and are ok in pe...
1                                  exactly as expected
2    this is a really nice long nightgown and is ve...
3    cool socks they are great gifts for my childre...
4    i received this item at a discount in exchange...
Name: review_body, dtype: object

spliting data

In [66]:
X_train, X_test, Y_train, Y_test = train_test_split(selected_reviews['review_body'].values,\
                                                    selected_reviews['label'].values,\
                                                    test_size=0.30,\
                                                    shuffle=False)

In [67]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words= 50000)
tokenizer.fit_on_texts(X_train)

In [68]:
X_train_tok = tokenizer.texts_to_sequences(X_train)
X_test_tok = tokenizer.texts_to_sequences(X_test)

In [78]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,SimpleRNN, GRU

In [70]:
max_review_length = 600
X_train_pad = sequence.pad_sequences(X_train_tok, maxlen=max_review_length)
X_test_pad = sequence.pad_sequences(X_test_tok, maxlen=max_review_length)

In [71]:
embedding_vector_length = 32
top_words = 50000
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 600, 32)           1600000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 1,653,301
Trainable params: 1,653,301
Non-trainable params: 0
_________________________________________________________________
None


In [72]:
history = model.fit(X_train_pad, Y_train, nb_epoch=10, batch_size=64,validation_data=(X_test_pad,Y_test))
# Final evaluation of the model on test data
scores = model.evaluate(X_test_pad, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 2455 samples, validate on 1053 samples
Epoch 1/10
2455/2455 [==============================] - 32s 13ms/sample - loss: 0.6439 - accuracy: 0.6578 - val_loss: 0.6023 - val_accuracy: 0.6667
Epoch 2/10
2455/2455 [==============================] - 31s 12ms/sample - loss: 0.5242 - accuracy: 0.7389 - val_loss: 0.5451 - val_accuracy: 0.7303
Epoch 3/10
2455/2455 [==============================] - 30s 12ms/sample - loss: 0.4691 - accuracy: 0.7959 - val_loss: 1.0400 - val_accuracy: 0.4938
Epoch 4/10
2455/2455 [==============================] - 30s 12ms/sample - loss: 0.4545 - accuracy: 0.7980 - val_loss: 0.5975 - val_accuracy: 0.7512
Epoch 5/10
2455/2455 [==============================] - 30s 12ms/sample - loss: 0.5750 - accuracy: 0.7548 - val_loss: 0.5766 - val_accuracy: 0.6914
Epoch 6/10
2455/2455 [==============================] - 31s 12ms/sample - loss: 0.3580 - accuracy: 0.8464 - val_loss: 0.6291 - val_accuracy: 0.7322
Epoch 7/10
2455/2455 [==============================] - 31s 12ms

In [75]:
top_words=50000
embedding_vector_length = 40
model = Sequential()
model.add(Embedding(top_words+1, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(16))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 600, 40)           2000040   
_________________________________________________________________
lstm_5 (LSTM)                (None, 600, 32)           9344      
_________________________________________________________________
lstm_6 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 2,012,537
Trainable params: 2,012,537
Non-trainable params: 0
_________________________________________________________________
None


In [76]:
history = model.fit(X_train_pad, Y_train, nb_epoch=10, batch_size=64,validation_data=(X_test_pad,Y_test))
# Final evaluation of the model on test data
scores = model.evaluate(X_test_pad, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 2455 samples, validate on 1053 samples
Epoch 1/10
2455/2455 [==============================] - 29s 12ms/sample - loss: 0.6511 - accuracy: 0.6574 - val_loss: 0.6121 - val_accuracy: 0.6420
Epoch 2/10
2455/2455 [==============================] - 24s 10ms/sample - loss: 0.5067 - accuracy: 0.7523 - val_loss: 0.5326 - val_accuracy: 0.7227
Epoch 3/10
2455/2455 [==============================] - 24s 10ms/sample - loss: 0.3894 - accuracy: 0.8310 - val_loss: 0.5165 - val_accuracy: 0.7426
Epoch 4/10
2455/2455 [==============================] - 24s 10ms/sample - loss: 0.3141 - accuracy: 0.8741 - val_loss: 0.5620 - val_accuracy: 0.7464
Epoch 5/10
2455/2455 [==============================] - 23s 9ms/sample - loss: 0.2780 - accuracy: 0.8961 - val_loss: 0.6382 - val_accuracy: 0.7379
Epoch 6/10
2455/2455 [==============================] - 24s 10ms/sample - loss: 0.2156 - accuracy: 0.9206 - val_loss: 0.6207 - val_accuracy: 0.7455
Epoch 7/10
2455/2455 [==============================] - 24s 10ms/

In [86]:
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(GRU(256, return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(SimpleRNN(128))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary() 

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 600, 40)           2000000   
_________________________________________________________________
gru_4 (GRU)                  (None, 600, 256)          228864    
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 128)               49280     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
Total params: 2,278,273
Trainable params: 2,278,273
Non-trainable params: 0
_________________________________________________________________


In [87]:
history = model.fit(X_train_pad, Y_train, nb_epoch=10, batch_size=64,validation_data=(X_test_pad,Y_test))
# Final evaluation of the model on test data
scores = model.evaluate(X_test_pad, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 2455 samples, validate on 1053 samples
Epoch 1/10
2455/2455 [==============================] - 83s 34ms/sample - loss: 0.6322 - accuracy: 0.6354 - val_loss: 0.6001 - val_accuracy: 0.6781
Epoch 2/10
2455/2455 [==============================] - 79s 32ms/sample - loss: 0.5551 - accuracy: 0.7210 - val_loss: 0.5783 - val_accuracy: 0.6876
Epoch 3/10
2455/2455 [==============================] - 78s 32ms/sample - loss: 0.4078 - accuracy: 0.8179 - val_loss: 0.5905 - val_accuracy: 0.7056
Epoch 4/10
2455/2455 [==============================] - 79s 32ms/sample - loss: 0.2863 - accuracy: 0.8851 - val_loss: 0.6428 - val_accuracy: 0.6952
Epoch 5/10
2455/2455 [==============================] - 79s 32ms/sample - loss: 0.2164 - accuracy: 0.9181 - val_loss: 0.9627 - val_accuracy: 0.6116
Epoch 6/10
2455/2455 [==============================] - 78s 32ms/sample - loss: 0.1211 - accuracy: 0.9552 - val_loss: 0.9403 - val_accuracy: 0.6610
Epoch 8/10
2455/2455 [==============================] - 79s 32ms